In [3]:
import pandas as pd
import numpy as np

# 날짜 생성 (2020-01-01 ~ 2025-03-31, 하루 단위)
dates = pd.date_range(start="2020-01-01", end="2025-03-31", freq="D")

# 가상의 전력 사용량 데이터 (200~500 kWh 사이 난수)
usage = np.random.randint(200, 500, size=len(dates))

# 가상의 요금 데이터 (기본요금 + 사용량 기반 요금)
price = 5000 + usage * np.random.uniform(80, 120)  # kWh당 80~120원

# 데이터프레임 생성
df = pd.DataFrame({
    "date": dates,
    "usage": usage,
    "price": price.astype(int)  # 소수점 제거
})

# CSV 저장
file_path = "electricity_data.csv"
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print(f"CSV 파일이 생성되었습니다: {file_path}")
print(df.head())


CSV 파일이 생성되었습니다: electricity_data.csv
        date  usage  price
0 2020-01-01    338  42298
1 2020-01-02    244  31925
2 2020-01-03    321  40422
3 2020-01-04    293  37332
4 2020-01-05    342  42739


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# ----------------------------------------
# 1. CSV 데이터 불러오기
# ----------------------------------------
file_path = "electricity_data.csv"  # CSV 파일 경로
df = pd.read_csv(file_path, parse_dates=["date"])
df = pd.read_csv(file_path, encoding="utf-8")  # 또는 encoding="cp949"

# 결측치 제거
df = df.dropna()

# ----------------------------------------
# 2. 데이터 전처리
# ----------------------------------------
scaler = MinMaxScaler()
price_scaled = scaler.fit_transform(df["price"].values.reshape(-1, 1))

sequence_length = 10
X, y = [], []
for i in range(len(price_scaled) - sequence_length):
    X.append(price_scaled[i:i+sequence_length])
    y.append(price_scaled[i+sequence_length])
X, y = np.array(X), np.array(y)

# Train / Test split
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# ----------------------------------------
# 3. LSTM 모델 정의 & 학습
# ----------------------------------------
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=30, batch_size=16, verbose=1)

# ----------------------------------------
# 4. 예측
# ----------------------------------------
y_pred_scaled = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred_scaled)  # 스케일 되돌리기
y_test_real = scaler.inverse_transform(y_test)    # 실제 값도 되돌리기

# ----------------------------------------
# 5. 퍼센타일 구간 경계값 계산 (학습 데이터 기준)
# ----------------------------------------
boundaries = {
    "하위 25%": np.percentile(df["price"], 25),
    "하위 10%": np.percentile(df["price"], 10),
    "하위 5%" : np.percentile(df["price"], 5),
    "상위 25%": np.percentile(df["price"], 75),
    "상위 10%": np.percentile(df["price"], 90),
    "상위 5%" : np.percentile(df["price"], 95)
}

# 구간 분류 함수
def classify_price(price, bounds):
    if price >= bounds["상위 5%"]:
        return "상위 5%"
    elif price >= bounds["상위 10%"]:
        return "상위 10%"
    elif price >= bounds["상위 25%"]:
        return "상위 25%"
    elif price <= bounds["하위 5%"]:
        return "하위 5%"
    elif price <= bounds["하위 10%"]:
        return "하위 10%"
    elif price <= bounds["하위 25%"]:
        return "하위 25%"
    else:
        return "중간 구간"

# ----------------------------------------
# 6. 결과 출력
# ----------------------------------------
print("\n=== 예측 결과 샘플 ===")
for i in range(10):  # 예측 10개만 출력
    pred_price = y_pred[i][0]
    category = classify_price(pred_price, boundaries)
    print(f"예측 요금: {pred_price:,.0f}원 → {category}")


Epoch 1/30


c:\Users\Admin\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1931 
Epoch 2/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0879
Epoch 3/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0879
Epoch 4/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0864
Epoch 5/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0855
Epoch 6/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0861
Epoch 7/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0842
Epoch 8/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0858
Epoch 9/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0792
Epoch 10/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0834
Epoch 11/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0849
Epoch 12/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0866
Epoch 13/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0847
Epoch 14/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0847
Epoch 15/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0845
Epoch 16/30
96/96 ━━━━━━━━━━━